# Object Detection Using Yolov3

## Importing Libraries

In [322]:
import cv2
import numpy as np

## Loading Weights and Classes from Pre-trained model

In [323]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg.txt")
classes = []
with open("coco.names.txt", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

## Reading image and resizing it

In [324]:
# img = cv2.imread("Images/granada-transport.jpg")
# img = cv2.imread("Images/peoplemoving.jpg")
# img = cv2.imread("Images/2019_11_19_pedestrians_crossing_stopped_cars_wet_city_street_at_dusk_WP.jpg")
# img = cv2.imread("Images/blind-person-senses-01.jpg")
# img = cv2.imread("Images/image.jpg")
# img = cv2.imread("Images/IMG-20200327-WA0079.jpg")
# img = cv2.imread("Images/images.jpg")
# img = cv2.imread("Images/ba84726c27ebdaba6e7aca46353f360c.jpg")
# img = cv2.imread("Images/Elephant-IndianRoller2.jpg")
# img = cv2.imread("Images/GettyImages-944229640-7bc4b13a94684b758c0b4450e7cdafb1.jpg")
# img = cv2.imread("Images/maxresdefault.jpg")
# img = cv2.imread("Images/Fanjeaux_018_20140722-Edit_Cycle_-cycling_-France_-Group_-TDF_-Tour-de-France.jpg")
# img = cv2.imread("Images/images1111.jpg")
# img = cv2.imread("Images/Can-You-Use-a-TV-as-a-Computer-Monitor-1280x720.jpg")
img = cv2.imread("Images/US_Navy_050210-N-2802K-001.jpg")

img = cv2.resize(img, None, fx=0.5, fy=0.5)
height, width, channels = img.shape

## Detecting objects

In [325]:
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

## Showing informations on the screen

In [326]:
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

## Performing Non Max Suppresion

In [327]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

## Extracting information and showing it on screen

In [328]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 2, color, 2)


cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()